In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.colab import drive
drive.mount('/content/drive')
train_csv_path = "/content/drive/MyDrive/UIUC-CS/CS412/Final Project/train.csv"
#train_csv_path = '/kaggle/input/bigquery-geotab-intersection-congestion/train.csv'
df = pd.read_csv(train_csv_path)
print(df.head())

Mounted at /content/drive
     RowId  IntersectionId   Latitude  Longitude  \
0  1921357               0  33.791659 -84.430032   
1  1921358               0  33.791659 -84.430032   
2  1921359               0  33.791659 -84.430032   
3  1921360               0  33.791659 -84.430032   
4  1921361               0  33.791659 -84.430032   

                EntryStreetName                ExitStreetName EntryHeading  \
0  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
1  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
2  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
3  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
4  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   

  ExitHeading  Hour  Weekend  ...  TimeFromFirstStop_p40  \
0          NW     0        0  ...                    0.0   
1          SE     0        0  ...                    0.0   
2          N

In [3]:
df = pd.read_csv(train_csv_path)
x_train = df[['IntersectionId','EntryHeading','ExitHeading']]
x_train = pd.concat([x_train,pd.get_dummies(df["City"],dummy_na=False, drop_first=False, dtype=int)],axis=1)
print(x_train.head())
heading_map = {'N':1,'NE':2,'E':3,'SE':4,'S':5, 'SW':6, 'W':7, 'NW': 8}
x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)
x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)
x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)
x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)
print(x_train)


   IntersectionId EntryHeading ExitHeading  Atlanta  Boston  Chicago  \
0               0           NW          NW        1       0        0   
1               0           SE          SE        1       0        0   
2               0           NW          NW        1       0        0   
3               0           SE          SE        1       0        0   
4               0           NW          NW        1       0        0   

   Philadelphia  
0             0  
1             0  
2             0  
3             0  
4             0  


/tmp/ipython-input-1820002226.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)


        IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0                    0             8            8        1       0        0   
1                    0             4            4        1       0        0   
2                    0             8            8        1       0        0   
3                    0             4            4        1       0        0   
4                    0             8            8        1       0        0   
...                ...           ...          ...      ...     ...      ...   
856382            1990             5            7        0       0        0   
856383            1990             5            5        0       0        0   
856384            1990             5            5        0       0        0   
856385            1990             7            7        0       0        0   
856386            1990             5            5        0       0        0   

        Philadelphia  
0                  0  
1    

/tmp/ipython-input-1820002226.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)


In [4]:
# Hour: 0-23 cyclical -> [sin(2pi*hour/24), cos(2pi*hour/24)]
df['hour_sin'] = np.sin(2*np.pi*df['Hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['Hour']/24)
print(df.head())
print(df['Month'].unique())

df['month_sin'] = np.sin(2*np.pi*df['Month']/12)
df['month_cos'] = np.cos(2*np.pi*df['Month']/12)
print(df.head())

     RowId  IntersectionId   Latitude  Longitude  \
0  1921357               0  33.791659 -84.430032   
1  1921358               0  33.791659 -84.430032   
2  1921359               0  33.791659 -84.430032   
3  1921360               0  33.791659 -84.430032   
4  1921361               0  33.791659 -84.430032   

                EntryStreetName                ExitStreetName EntryHeading  \
0  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
1  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
2  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
3  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
4  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   

  ExitHeading  Hour  Weekend  ...  TimeFromFirstStop_p60  \
0          NW     0        0  ...                    0.0   
1          SE     0        0  ...                    0.0   
2          NW     1        0  ...     

In [5]:
x_train['hour_sin'] = df['hour_sin']
x_train['hour_cos'] = df['hour_cos']
x_train['month_sin'] = df['month_sin']
x_train['month_cos'] = df['month_cos']
x_train['Weekend'] = df['Weekend']
x_train['Month'] = df['Month']
x_train['Hour'] = df['Hour']
print(x_train)

        IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0                    0             8            8        1       0        0   
1                    0             4            4        1       0        0   
2                    0             8            8        1       0        0   
3                    0             4            4        1       0        0   
4                    0             8            8        1       0        0   
...                ...           ...          ...      ...     ...      ...   
856382            1990             5            7        0       0        0   
856383            1990             5            5        0       0        0   
856384            1990             5            5        0       0        0   
856385            1990             7            7        0       0        0   
856386            1990             5            5        0       0        0   

        Philadelphia  hour_sin      hour_cos     mo

In [6]:
y_train = pd.DataFrame()
y_train['TotalTimeStopped_p20'] = df['TotalTimeStopped_p20']
y_train['TotalTimeStopped_p50'] = df['TotalTimeStopped_p50']
y_train['TotalTimeStopped_p80'] = df['TotalTimeStopped_p80']
y_train['DistanceToFirstStop_p20'] = df['DistanceToFirstStop_p20']
y_train['DistanceToFirstStop_p50'] = df['DistanceToFirstStop_p50']
y_train['DistanceToFirstStop_p80'] = df['DistanceToFirstStop_p80']
print(y_train.head())

   TotalTimeStopped_p20  TotalTimeStopped_p50  TotalTimeStopped_p80  \
0                   0.0                   0.0                   0.0   
1                   0.0                   0.0                   0.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   DistanceToFirstStop_p20  DistanceToFirstStop_p50  DistanceToFirstStop_p80  
0                      0.0                      0.0                      0.0  
1                      0.0                      0.0                      0.0  
2                      0.0                      0.0                      0.0  
3                      0.0                      0.0                      0.0  
4                      0.0                      0.0                      0.0  


In [7]:
#Split data into train and val
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
print(x_train.head())
print("len x_train", len(x_train))
print("len x_val",len(x_val))
print(y_train.head())
print("len y_train", len(y_train))
print("len y_val",len(y_val))

        IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
538730             345             7            7        0       0        0   
630597             826             2            2        0       0        0   
817258            1796             6            6        0       0        0   
526197             264             5            5        0       0        0   
256532             667             2            3        0       1        0   

        Philadelphia      hour_sin  hour_cos     month_sin  month_cos  \
538730             1 -7.071068e-01 -0.707107 -8.660254e-01   0.500000   
630597             1  9.659258e-01 -0.258819 -8.660254e-01  -0.500000   
817258             1  7.071068e-01 -0.707107 -5.000000e-01   0.866025   
526197             1  5.000000e-01 -0.866025 -2.449294e-16   1.000000   
256532             0  1.224647e-16 -1.000000 -2.449294e-16   1.000000   

        Weekend  Month  Hour  
538730        0     10    15  
630597        0      8  

Improved LSTM - add Gating Rule

lightGBM for 0 and 1

In [8]:
# probability label, same as before
y_train_prob = (((y_train['TotalTimeStopped_p50'] > 0) |
                 (y_train['DistanceToFirstStop_p50'] > 0))).astype(float)

y_val_prob = (((y_val['TotalTimeStopped_p50'] > 0) |
               (y_val['DistanceToFirstStop_p50'] > 0))).astype(float)


In [9]:
# Month one-hot
train_month_oh = pd.get_dummies(x_train['Month'], prefix='M')
val_month_oh   = pd.get_dummies(x_val['Month'], prefix='M')

train_month_oh, val_month_oh = train_month_oh.align(val_month_oh, axis=1, fill_value=0)

# city one-hot
city_cols = [c for c in ["Atlanta","Boston","Chicago","Philadelphia"] if c in x_train.columns]

# numeric columns
num_cols = [c for c in x_train.columns if c not in city_cols + ['Month']]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_num = scaler.fit_transform(x_train[num_cols])
x_val_num   = scaler.transform(x_val[num_cols])

# final training matrix
X_train_full = np.hstack([train_month_oh.values,
                          x_train[city_cols].values,
                          x_train_num])

X_val_full = np.hstack([val_month_oh.values,
                        x_val[city_cols].values,
                        x_val_num])


In [11]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error, accuracy_score

# LightGBM dataset
train_ds = lgb.Dataset(X_train_full, label=y_train_prob)
val_ds   = lgb.Dataset(X_val_full,   label=y_val_prob)

params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'num_leaves': 64,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'min_data_in_leaf': 50,
    'verbose': -1
}

model = lgb.train(
    params,
    train_ds,
    num_boost_round=2000,
    valid_sets=[train_ds, val_ds],
    valid_names=['train', 'val'],
)

# Predict probability on validation
pred_val = model.predict(X_val_full, num_iteration=model.best_iteration)
pred_val = np.clip(pred_val, 0, 1)

acc  = accuracy_score(y_val_prob, (pred_val > 0.5))
rmse = np.sqrt(mean_squared_error(y_val_prob, pred_val))

print("=== LightGBM Probability ===")
print("Accuracy:", round(acc,4))
print("RMSE   :", round(rmse,4))


=== LightGBM Probability ===
Accuracy: 0.7795
RMSE   : 0.3906


LSTM improved

In [12]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

# 6 target col
target_cols = [
    'TotalTimeStopped_p20',
    'TotalTimeStopped_p50',
    'TotalTimeStopped_p80',
    'DistanceToFirstStop_p20',
    'DistanceToFirstStop_p50',
    'DistanceToFirstStop_p80'
]

if isinstance(X_train_full, np.ndarray):
    X_train_full = pd.DataFrame(X_train_full, index=x_train.index)
if isinstance(X_val_full, np.ndarray):
    X_val_full = pd.DataFrame(X_val_full, index=x_val.index)


In [14]:
def index_X_full(X_full: pd.DataFrame, idx_sorted):
    return X_full.loc[idx_sorted].to_numpy(np.float32)


city_cols = [c for c in ['Atlanta','Boston','Chicago','Philadelphia'] if c in x_train.columns]

def prepare_sorted_data_for_lstm(X_raw, X_full, y_df, city_cols, target_cols):
    tmp = X_raw.copy()
    tmp['CityKey'] = tmp[city_cols].idxmax(axis=1)
    tmp['IntersectionId'] = df.loc[tmp.index, 'IntersectionId']

    # sort
    tmp_sorted = tmp.sort_values(['CityKey','IntersectionId','Month','Hour'])

    idx_sorted = tmp_sorted.index.to_numpy()

    X_np = index_X_full(X_full, idx_sorted)
    y_np = y_df.loc[idx_sorted, target_cols].to_numpy(np.float32)

    city_np  = tmp_sorted['CityKey'].to_numpy()
    inter_np = tmp_sorted['IntersectionId'].to_numpy()
    month_np = tmp_sorted['Month'].to_numpy()

    return X_np, y_np, city_np, inter_np, month_np, idx_sorted


In [15]:
def get_groups(city_np, inter_np, month_np):
    key = city_np.astype(str) + "_" + inter_np.astype(str) + "_" + month_np.astype(str)
    cut = np.where(key[1:] != key[:-1])[0] + 1
    return np.split(np.arange(len(key)), cut)


In [16]:
def make_sequences_numpy_with_index(X_np, y_np, idx_sorted, groups, seq_len):

    nfeat     = X_np.shape[1]
    n_targets = y_np.shape[1]

    est = sum(max(0, len(g) - seq_len + 1) for g in groups)

    X_out   = np.empty((est, seq_len, nfeat),   dtype=np.float32)
    y_out   = np.empty((est, n_targets),        dtype=np.float32)
    idx_out = np.empty((est,),                  dtype=idx_sorted.dtype)

    ptr = 0

    for g in groups:
        L = len(g)
        if L < seq_len:
            continue

        arrX = X_np[g]
        arrY = y_np[g]
        arrI = idx_sorted[g]

        subX = np.lib.stride_tricks.as_strided(
            arrX,
            shape=(L - seq_len + 1, seq_len, nfeat),
            strides=(arrX.strides[0], arrX.strides[0], arrX.strides[1])
        )

        nseq = subX.shape[0]

        X_out[ptr:ptr+nseq]   = subX
        y_out[ptr:ptr+nseq]   = arrY[seq_len-1:]
        idx_out[ptr:ptr+nseq] = arrI[seq_len-1:]

        ptr += nseq

    return X_out[:ptr], y_out[:ptr], idx_out[:ptr]


In [17]:
SEQ_LEN = 12

def build_lstm_sequences(X_raw, X_full, y_df, seq_len, city_cols, target_cols):
    X_np, y_np, city_np, inter_np, month_np, idx_sorted = prepare_sorted_data_for_lstm(
        X_raw, X_full, y_df, city_cols, target_cols
    )
    groups = get_groups(city_np, inter_np, month_np)
    return make_sequences_numpy_with_index(X_np, y_np, idx_sorted, groups, seq_len)

# Train sequences
Xtr_seq, ytr_seq, idx_tr_seq = build_lstm_sequences(
    x_train, X_train_full, y_train, SEQ_LEN, city_cols, target_cols
)

# Val sequences
Xva_seq, yva_seq, idx_va_seq = build_lstm_sequences(
    x_val,   X_val_full,   y_val,   SEQ_LEN, city_cols, target_cols
)

print("Train seq:", Xtr_seq.shape, ytr_seq.shape)
print("Val   seq:", Xva_seq.shape, yva_seq.shape)


Train seq: (409983, 12, 22) (409983, 6)
Val   seq: (29397, 12, 22) (29397, 6)


In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_lstm_regressor(seq_len, feat_dim, out_dim=6):
    model = keras.Sequential([
        layers.Input((seq_len, feat_dim)),
        layers.LSTM(128),
        layers.Dense(64, activation='relu'),
        layers.Dense(out_dim)   # 6 target
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss='mse'
    )
    return model

reg_lstm = build_lstm_regressor(SEQ_LEN, Xtr_seq.shape[2], out_dim=len(target_cols))
reg_lstm.summary()

reg_lstm.fit(
    Xtr_seq, ytr_seq,
    validation_data=(Xva_seq, yva_seq),
    batch_size=256,
    epochs=5,
    callbacks=[keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)],
    verbose=1
)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128)            │        77,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 85,958 (335.77 KB)

 Trainable params: 85,958 (335.77 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1602/1602 ━━━━━━━━━━━━━━━━━━━━ 110s 66ms/step - loss: 7165.8994 - val_loss: 10396.5820
Epoch 2/5
1602/1602 ━━━━━━━━━━━━━━━━━━━━ 105s 66ms/step - loss: 6624.5479 - val_loss: 10182.8447
Epoch 3/5
1602/1602 ━━━━━━━━━━━━━━━━━━━━ 105s 66ms/step - loss: 6504.8457 - val_loss: 10397.1240
Epoch 4/5
1602/1602 ━━━━━━━━━━━━━━━━━━━━ 109s 68ms/step - loss: 6530.3896 - val_loss: 10348.2959
Epoch 5/5
1602/1602 ━━━━━━━━━━━━━━━━━━━━ 117s 73ms/step - loss: 6523.6299 - val_loss: 10356.7383


predict and eval

In [20]:
X_val_for_seq = X_val_full.loc[idx_va_seq]

# LightGBM predict
prob_seq = model.predict(X_val_for_seq)  # shape: (N_seq,)
prob_seq = np.clip(prob_seq, 0, 1)

In [21]:
# LSTM predict
y_pred_lstm = reg_lstm.predict(Xva_seq, batch_size=256)  # shape: (N_seq, 6)

115/115 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step


In [22]:
threshold = 0.5

mask = (prob_seq > threshold).astype(np.float32)  # (N_seq,)
mask = mask[:, None]

# final predictions
y_pred_final = y_pred_lstm * mask   # prob<=0.5 → 全 0


In [30]:
rmse_two_stage = {}

for i, col in enumerate(target_cols):
    rmse = np.sqrt(mean_squared_error(yva_seq[:, i], y_pred_final[:, i]))
    rmse_two_stage[col] = rmse

print("\n=== RMSE ===")
for k, v in rmse_two_stage.items():
    print(k.ljust(35), ":", round(v, 4))
print("\nOverall RMSE:", round(np.sqrt(np.mean((yva_seq - y_pred_final)**2)), 4))



=== RMSE ===
TotalTimeStopped_p20                : 6.6729
TotalTimeStopped_p50                : 13.8735
TotalTimeStopped_p80                : 24.5145
DistanceToFirstStop_p20             : 29.4527
DistanceToFirstStop_p50             : 67.9417
DistanceToFirstStop_p80             : 137.0064

Overall RMSE: 64.6687
